In [1]:
import math
import pandas as pd
import numpy as np
from scipy.sparse import hstack

In [2]:
df=pd.read_csv('features.csv')

In [3]:
df.head()

,match_id,start_time,lobby_type,r1_hero,r1_level,r1_xp,r1_gold,r1_lh,r1_kills,r1_deaths,...,dire_boots_count,dire_ward_observer_count,dire_ward_sentry_count,dire_first_ward_time,duration,radiant_win,tower_status_radiant,tower_status_dire,barracks_status_radiant,barracks_status_dire
0,0,1430198770,7,11,5,2098,1489,20,0,0,...,4,2,2,-52.0,2874,1,1796,0,51,0
1,1,1430220345,0,42,4,1188,1033,9,0,1,...,4,3,1,-5.0,2463,1,1974,0,63,1
2,2,1430227081,7,33,4,1319,1270,22,0,0,...,4,3,1,13.0,2130,0,0,1830,0,63
3,3,1430263531,1,29,4,1779,1056,14,0,0,...,4,2,0,27.0,1459,0,1920,2047,50,63
4,4,1430282290,7,13,4,1431,1090,8,1,0,...,3,3,0,-16.0,2449,0,4,1974,3,63


In [4]:
df.shape

(97230, 109)

In [5]:
# Delete features from feature
features = df.drop(['tower_status_radiant', 'tower_status_dire', 
                    'barracks_status_radiant', 'barracks_status_dire', 'duration','radiant_win'], axis=1)

In [6]:
features.shape

(97230, 103)

In [7]:
# Count Nan
#features.count() 

In [8]:
# Replaced Nan by 0
features.fillna(0, inplace=True)

In [9]:
features.head()

,match_id,start_time,lobby_type,r1_hero,r1_level,r1_xp,r1_gold,r1_lh,r1_kills,r1_deaths,...,radiant_ward_sentry_count,radiant_first_ward_time,dire_bottle_time,dire_courier_time,dire_flying_courier_time,dire_tpscroll_count,dire_boots_count,dire_ward_observer_count,dire_ward_sentry_count,dire_first_ward_time
0,0,1430198770,7,11,5,2098,1489,20,0,0,...,0,35.0,103.0,-84.0,221.0,3,4,2,2,-52.0
1,1,1430220345,0,42,4,1188,1033,9,0,1,...,0,-20.0,149.0,-84.0,195.0,5,4,3,1,-5.0
2,2,1430227081,7,33,4,1319,1270,22,0,0,...,1,-39.0,45.0,-77.0,221.0,3,4,3,1,13.0
3,3,1430263531,1,29,4,1779,1056,14,0,0,...,0,-30.0,124.0,-80.0,184.0,0,4,2,0,27.0
4,4,1430282290,7,13,4,1431,1090,8,1,0,...,0,46.0,182.0,-80.0,225.0,6,3,3,0,-16.0


## Model train

In [10]:
y = df['radiant_win']
X = features.drop(['match_id','start_time','lobby_type', 'r1_hero', 'r2_hero', 'r3_hero','r4_hero', 'r5_hero',
                          'd2_hero', 'd3_hero', 'd4_hero', 'd5_hero'], axis = 1)

In [11]:
from sklearn.model_selection import KFold, train_test_split, cross_val_score
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn.svm import SVC
from sklearn.metrics import roc_auc_score

In [12]:
X['mean_1_r'] = np.mean([X['r1_level'],X['r1_xp'], X['r1_gold'],X['r1_lh'],X['r1_items'], X['r1_deaths']], axis = 0)
X['mean_2_r'] = np.mean([X['radiant_bottle_time'], X['radiant_courier_time'],
                         X['radiant_flying_courier_time'], X['radiant_tpscroll_count'],
                         X['radiant_boots_count'], X['radiant_ward_observer_count'], X['radiant_ward_sentry_count'], 
                         X['radiant_first_ward_time']], axis=0)
#['sum_1_r'] = sum([X['r1_level'],X['r1_xp'], X['r1_gold'],X['r1_lh'],X['r1_items'], X['r1_deaths']])
#X['sum_2_r'] = sum([X['radiant_bottle_time'], X['radiant_courier_time'],
#                       X['radiant_flying_courier_time'], X['radiant_tpscroll_count'],
 #                     X['radiant_boots_count'], X['radiant_ward_observer_count'], X['radiant_ward_sentry_count'], 
 #                     X['radiant_first_ward_time']])

In [13]:
#X['mean_1_d'] = np.mean([X['d1_level'],X['d1_xp'], X['d1_gold'],X['d1_lh'],X['d1_items'], X['d1_deaths']], axis = 0)
#X['mean_2_r'] = np.mean([X['dire_bottle_time'], X['dire_courier_time'], X['dire_flying_courier_time'], 
#                         X['dire_tpscroll_count'], X['dire_boots_count'], X['dire_ward_observer_count'],
#                         X['dire_ward_sentry_count'], X['dire_first_ward_time']], axis=0)
#X['sum_d1_level_d1_xp_d1_gold'] = sum([X['d1_level'],X['d1_xp'], X['d1_gold']])

In [14]:
# N — количество различных героев в выборке
N = 112
X_pick = np.zeros((features.shape[0], N))

for i, match_id in enumerate(features.index):
    for p in range(5):
        X_pick[i, features.ix[match_id, 'r%d_hero' % (p+1)]-1] = 1
        X_pick[i, features.ix[match_id, 'd%d_hero' % (p+1)]-1] = -1


C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:7: DeprecationWarning: 
.ix is deprecated. Please use
.loc for label based indexing or
.iloc for positional indexing

See the documentation here:
http://pandas.pydata.org/pandas-docs/stable/indexing.html#ix-indexer-is-deprecated
  import sys
C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:8: DeprecationWarning: 
.ix is deprecated. Please use
.loc for label based indexing or
.iloc for positional indexing

See the documentation here:
http://pandas.pydata.org/pandas-docs/stable/indexing.html#ix-indexer-is-deprecated
  


In [15]:
X4 = hstack([X, X_pick])

In [16]:
X4.shape

(97230, 205)

### LogisticRegression

In [17]:
%%time
kf = KFold(n_splits=5, shuffle=True, random_state=0) 
results = pd.DataFrame()             
i=1
for trainIdx, testIdx in kf.split(X4, y):
    logres_l2 = LogisticRegression(C=1, penalty='l2') 
    logres_l2.fit(X4.tocsc()[trainIdx, :], y[trainIdx]) 
    pred_log = logres_l2.predict_proba(X4.tocsc()[testIdx, :])[:, 1] 
    results.loc[i,'test_AUC C=1'] = roc_auc_score(y[testIdx], pred_log) 
    i+=1

Wall time: 4min 36s


In [18]:
print(results)
print("Mean ROC-AUC LogisticRegression: ")
print(np.mean(results))

   test_AUC C=1
1      0.750800
2      0.753715
3      0.749884
4      0.754012
5      0.749664
Mean ROC-AUC LogisticRegression: 
test_AUC C=1    0.751615
dtype: float64


In [ ]:
'''
0.751604 нульовий варіант
0.751558  X['r1_level'],X['r1_xp'], X['r1_gold'],X['r1_lh'],X['r1_items'], X['r1_deaths'], X['r1_kills']
0.751615  X['mean_1_r']+X['mean_2_r']
0.751576  X['mean_1_']+X['mean_2_r'] + X['mean_1_d']+X['mean_2_d']
0.751606 X['mean_1_']+X['mean_2_r'] + X['mean_1_d']+X['mean_2_d'] + X['sum_r1_level_r1_xp_r1_gold']
0.751588  X['mean_1_']+X['mean_2_r']+ X['mean_1_d']+X['mean_2_d']+ X['sum_r1_level_r1_xp_r1_gold']+X['sum_d1_level_d1_xp_d1_gold']
0.751579  X['mean_1_r']+X['mean_2_r']+ X['sum_r1_level_r1_xp_r1_gold']
'''

### RandomForestClassifier

In [ ]:
%%time
kf = KFold(n_splits=5, shuffle=True, random_state=0) 
results = pd.DataFrame()            
i=1
for trainIdx, testIdx in kf.split(X4, y):
    forest =  RandomForestClassifier(n_estimators=250, max_depth=15, random_state=0)
    forest.fit(X4.tocsc()[trainIdx,:], y[trainIdx]) 
    pred_forest = forest.predict_proba(X4.tocsc()[testIdx,:])[:, 1] 
    results.loc[i,'test_AUC n_estimators=200, max_depth=15'] = roc_auc_score(y[testIdx], pred_forest) 
    i+=1

In [ ]:
print(results)
print("Mean ROC-AUC RandomForestClassifier: ")
print(np.mean(results, axis=0))

### DecisionTreeClassifier

In [ ]:
%%time
kf = KFold(n_splits=5, shuffle=True, random_state=0) 
results = pd.DataFrame()            
i=1
for trainIdx, testIdx in kf.split(X4, y):
    tree =  DecisionTreeClassifier(max_depth=7, random_state=0)
    tree.fit(X4.tocsc()[trainIdx,:], y[trainIdx]) 
    pred_tree = tree.predict_proba(X4.tocsc()[testIdx,:])[:, 1] 
    results.loc[i,'test_AUC j=7'] = roc_auc_score(y[testIdx], pred_tree) 
    i+=1

In [ ]:
print("Mean ROC-AUC DecisionTreeClassifier:")
print(np.mean(results, axis=0))

### GradientBoostingClassifier

### SVC

### KNeighborsClassifier

### Feature_test

In [31]:
features_test=pd.read_csv('features_test.csv')

In [32]:
features_test.shape

(17177, 103)

In [33]:
features_test.head()

,match_id,start_time,lobby_type,r1_hero,r1_level,r1_xp,r1_gold,r1_lh,r1_kills,r1_deaths,...,radiant_ward_sentry_count,radiant_first_ward_time,dire_bottle_time,dire_courier_time,dire_flying_courier_time,dire_tpscroll_count,dire_boots_count,dire_ward_observer_count,dire_ward_sentry_count,dire_first_ward_time
0,6,1430287923,0,93,4,1103,1089,8,0,1,...,0,12.0,247.0,-86.0,272.0,3,4,2,0,118.0
1,7,1430293357,1,20,2,556,570,1,0,0,...,2,-29.0,168.0,-54.0,NaN,3,2,2,1,16.0
2,10,1430301774,1,112,2,751,808,1,0,0,...,1,-22.0,46.0,-87.0,186.0,1,3,3,0,-34.0
3,13,1430323933,1,27,3,708,903,1,1,1,...,2,-49.0,30.0,-89.0,210.0,3,4,2,1,-26.0
4,16,1430331112,1,39,4,1259,661,4,0,0,...,0,36.0,180.0,-86.0,180.0,1,3,2,1,-33.0


In [34]:
# Replaced Nan by 0
features_test.fillna(0, inplace=True)

In [35]:
X_test = features_test.drop(['match_id','start_time'], axis = 1)

In [36]:
# Delete 11 categorial features_test
X_test = X_test.drop(['lobby_type', 'r1_hero', 'r2_hero', 'r3_hero','r4_hero', 'r5_hero',
                          'd2_hero', 'd3_hero', 'd4_hero', 'd5_hero'], axis = 1)

In [37]:
X_test['mean_1_r'] = np.mean([X_test['r1_level'],X_test['r1_xp'], X_test['r1_gold'],X_test['r1_lh'],
                              X_test['r1_items'], X_test['r1_deaths']], axis = 0)
X_test['mean_2_r'] = np.mean([X_test['radiant_bottle_time'], X_test['radiant_courier_time'],
                         X_test['radiant_flying_courier_time'], X_test['radiant_tpscroll_count'],
                         X_test['radiant_boots_count'], X_test['radiant_ward_observer_count'], 
                              X_test['radiant_ward_sentry_count'], X_test['radiant_first_ward_time']], axis=0)

In [38]:
# N — количество различных героев в выборке
N = 112
X_pick_test = np.zeros((features_test.shape[0], N))

for i, match_id in enumerate(features_test.index):
    for p in range(5):
        X_pick_test[i, features_test.ix[match_id, 'r%d_hero' % (p+1)]-1] = 1
        X_pick_test[i, features_test.ix[match_id, 'd%d_hero' % (p+1)]-1] = -1

C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:7: DeprecationWarning: 
.ix is deprecated. Please use
.loc for label based indexing or
.iloc for positional indexing

See the documentation here:
http://pandas.pydata.org/pandas-docs/stable/indexing.html#ix-indexer-is-deprecated
  import sys
C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:8: DeprecationWarning: 
.ix is deprecated. Please use
.loc for label based indexing or
.iloc for positional indexing

See the documentation here:
http://pandas.pydata.org/pandas-docs/stable/indexing.html#ix-indexer-is-deprecated
  


In [39]:
X_test.shape

(17177, 93)

In [40]:
X_test = hstack([X_test, X_pick_test])

In [41]:
test_prob = logres_l2.predict_proba(X_test) 

In [42]:
predict = pd.DataFrame({'match_id': features_test['match_id'], 'radiant_win' :test_prob[:,1]})

In [43]:
predict.head()

,match_id,radiant_win
0,6,0.835779
1,7,0.793948
2,10,0.202140
3,13,0.859714
4,16,0.255546


In [44]:
predict.to_csv("predict_logisticregression.csv", index=False)